In [2]:
import pandas as pd

df = pd.read_csv("CMSC320_FinalProj_Database.csv")
df

,YEAR,COUNTY,HOSPITAL,OSHPDID,Procedure/Condition,Risk Adjusted Mortality Rate,# of Deaths,# of Cases,Hospital Ratings,LONGITUDE,LATITUDE
0,2016,AAAA,STATEWIDE,None,AAA Repair Unruptured,1.3,30,2358,None,None,None
1,2016,AAAA,STATEWIDE,None,AMI,6.1,3178,52167,None,None,None
2,2016,AAAA,STATEWIDE,None,Acute Stroke,9.1,5482,60184,None,None,None
3,2016,AAAA,STATEWIDE,None,Acute Stroke Hemorrhagic,21.1,2580,12210,None,None,None
4,2016,AAAA,STATEWIDE,None,Acute Stroke Ischemic,5,2258,45141,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
57649,2022,Yuba,Adventist Health and Rideout,106580996,Heart Failure,3.6,23,712,As Expected,-121.593602,39.138805
57650,2022,Yuba,Adventist Health and Rideout,106580996,Hip Fracture,4,6,167,As Expected,-121.593602,39.138805
57651,2022,Yuba,Adventist Health and Rideout,106580996,PCI,4.3,7,146,As Expected,-121.593602,39.138805
57652,2022,Yuba,Adventist Health and Rideout,106580996,Pancreatic Resection,NaN,NaN,NaN,NaN,-121.593602,39.138805


In [3]:
df.shape

(57654, 11)

In [50]:
df['# of Deaths'] = pd.to_numeric(df['# of Deaths'], errors='coerce') # Converts invalid values to NaN
df['Risk Adjusted Mortality Rate'] = pd.to_numeric(df['# of Deaths'], errors='coerce').fillna(10).astype(int)
nan_deaths = df[df['# of Deaths'].isna()]
print(nan_deaths)

Empty DataFrame
Columns: [YEAR, COUNTY, HOSPITAL, OSHPDID, Procedure/Condition, Risk Adjusted Mortality Rate, # of Deaths, # of Cases, Hospital Ratings, LONGITUDE, LATITUDE]
Index: []


In [52]:
df['HOSPITAL'].value_counts().head()

STATEWIDE                           180
Memorial Hospital of Gardena        180
Palmdale Regional Medical Center    180
Madera Community Hospital           180
San Mateo Medical Center            180
Name: HOSPITAL, dtype: int64

In [41]:
print(df['# of Deaths'].mean()) # overall deaths avg
print(df[df['Procedure/Condition']=='Heart Failure']['# of Deaths'].mean()) # Find the mean death rate of the "Heart Failure" procedure

9.758750476983383
18.58287795992714


We want to find whether the total number of inpatient deaths in California's hospitals caused by a "Heart Failure" procedure is the same as the overall mean death rate for all procedures with $\alpha = 0.05$.
$ \\ H_0: \mu = 10$
$ \\ H_a: \mu > 10$

In [51]:
from scipy import stats
deaths = df[df['Procedure/Condition']=='Heart Failure']['# of Deaths']

# Hypothesis
pop_mean = 10

t_statistic, p_value = stats.ttest_1samp(a=deaths, popmean=pop_mean, alternative='greater')
print("P-value:", p_value)

P-value: 0.0006872618407070447


Since $p \approx 0.0007 < 0.05$, we reject $H_0$ and claim that the total number of inpatient deaths in California's hospitals caused by a "Heart Failure" procedure is significantly greater than the overall average of deaths ($10$).